# Aligning a structure to another

We use `align.alignto` to align a structure to another.

**Last updated:** December 2022

**Minimum version of MDAnalysis:** 1.0.0

**Packages required:**
    
* MDAnalysis (<a data-cite="michaud-agrawal_mdanalysis_2011" href="https://doi.org/10.1002/jcc.21787">Michaud-Agrawal *et al.*, 2011</a>, <a data-cite="gowers_mdanalysis_2016" href="https://doi.org/10.25080/Majora-629e541a-00e">Gowers *et al.*, 2016</a>)
* MDAnalysisTests
   
**Optional packages for molecular visualisation:**
    
* [nglview](http://nglviewer.org/nglview/latest) (<a data-cite="nguyen_nglviewinteractive_2018" href="https://doi.org/10.1093/bioinformatics/btx789">Nguyen *et al.*, 2018</a>)

Throughout this tutorial we will include cells for visualising Universes with the [NGLView](http://nglviewer.org/nglview/latest/api.html) library. However, these will be commented out, and we will show the expected images generated instead of the interactive widgets.

**See also**

* [Aligning a trajectory to a frame from another](aligning_trajectory.ipynb)
* [Aligning a trajectory to a frame from itself](aligning_trajectory_to_frame.ipynb)
* [RMSD](rmsd.ipynb)


<div class="alert alert-info">
    
**Note**

MDAnalysis implements RMSD calculation using the fast QCP algorithm (<a data-cite="theobald_rapid_2005" href="https://doi.org/10.1107/S0108767305015266">Theobald, 2005</a>) and a rotation matrix *R* that minimises the RMSD (<a data-cite="liu_fast_2009" href="https://doi.org/10.1002/jcc.21439">Liu *et al.*, 2009</a>). Please cite (<a data-cite="theobald_rapid_2005" href="https://doi.org/10.1107/S0108767305015266">Theobald, 2005</a>) and (<a data-cite="liu_fast_2009" href="https://doi.org/10.1002/jcc.21439">Liu *et al.*, 2009</a>) when using the ``MDAnalysis.analysis.align`` module in published work.

</div>



In [1]:
import MDAnalysis as mda
from MDAnalysis.analysis import align
from MDAnalysis.tests.datafiles import CRD, PSF, DCD, DCD2
import nglview as nv

import warnings
# suppress some MDAnalysis warnings about writing PDB files
warnings.filterwarnings('ignore')

## Loading files

The test files we will be working with here are trajectories of a adenylate kinase (AdK), a phosophotransferase enzyme. (<a data-cite="beckstein_zipping_2009" href="https://doi.org/10.1016/j.jmb.2009.09.009">Beckstein *et al.*, 2009</a>) The trajectories sample a transition from a closed to an open conformation.

In [2]:
adk_open = mda.Universe(CRD, DCD2)
adk_closed = mda.Universe(PSF, DCD)

In [3]:
# adk_open_view = nv.show_mdanalysis(adk_open)
# adk_open_view

<center>
<div style='width: 800px'>
    
![adk_open_view](images/aligning_structure_to_another-adk_open_view.gif)
    
</div>
</center>

In [4]:
# adk_closed_view = nv.show_mdanalysis(adk_closed)
# adk_closed_view

<center>
<div style='width: 800px'>
    
![adk_closed_view](images/aligning_structure_to_another-adk_closed_view.gif)
    
</div>
</center>

Currently, the proteins are not aligned to each other. The difference becomes even more obvious when the closed conformation is compared to the open. Below, we set `adk_open` to the last frame and see the relative positions of each protein in a merged Universe.

In [5]:
adk_open.trajectory[-1] # last frame
merged = mda.Merge(adk_open.atoms, adk_closed.atoms)

In [6]:
# merged_view = nv.show_mdanalysis(merged)
# merged_view

<center>
<div style='width: 800px'>
    
![merged_view](images/aligning_structure_to_another-merged_view.png)
    
</div>
</center>

## Aligning a structure with align.alignto

`alignto` ([API docs](https://docs.mdanalysis.org/stable/documentation_pages/analysis/align.html#MDAnalysis.analysis.align.alignto)) aligns the mobile AtomGroup to the target AtomGroup by minimising the [root mean square deviation (RMSD) between particle positions](rmsd.ipynb) (please see the linked notebook for an explanation of RMSD). It returns *(old_rmsd, new_rmsd)*. By default (`match_atoms=True`), it will attempt to match the atoms between the mobile and reference structures by mass.

In [7]:
rmsds = align.alignto(adk_open,  # mobile
                      adk_closed,  # reference
                      select='name CA', # selection to operate on
                      match_atoms=True) # whether to match atoms
print(rmsds)

(21.712154435976014, 6.817293751703919)


In [8]:
# aligned_view = nv.show_mdanalysis(mda.Merge(adk_open.atoms, adk_closed.atoms))
# aligned_view

<center>
<div style='width: 800px'>
    
![aligned_view](images/aligning_structure_to_another-aligned_view.png)
    
</div>
</center>

However, you may want to align to a structure that where there is not a clear match in particle mass. For example, you could be aligning the alpha-carbons of an atomistic protein to the backbone beads of a coarse-grained structure. Below, we use the somewhat contrived example of aligning 214 alpha-carbons to the first 214 atoms of the reference structure. In this case, we need to switch `match_atoms=False` or the alignment will error.

In [9]:
rmsds = align.alignto(adk_open.select_atoms('name CA'),  # mobile
                      adk_closed.atoms[:214],  # reference
                      select='all', # selection to operate on
                      match_atoms=False) # whether to match atoms
print(rmsds)

(18.991465038265208, 16.603704620787127)


In [10]:
# shifted_aligned_view = nv.show_mdanalysis(mda.Merge(adk_open.atoms, adk_closed.atoms))
# shifted_aligned_view

<center>
<div style='width: 800px'>
    
![shifted_aligned_view](images/aligning_structure_to_another-shifted_aligned_view.png)
    
</div>
</center>

When we align structures, positions are set temporarily. If we flip to the first frame of `adk_open` and back to the last frame, we can see that it has returned to its original location.

In [11]:
adk_open.trajectory[0] # set to first frame
adk_open.trajectory[-1] # set to last frame

< Timestep 101 >

In [12]:
# reset_view = nv.show_mdanalysis(mda.Merge(adk_open.atoms, adk_closed.atoms))
# reset_view

<center>
<div style='width: 800px'>
    
![reset_view](images/aligning_structure_to_another-reset_view.png)
    
</div>
</center>

You can save the aligned positions by writing them out to a PDB file and creating a new Universe.

In [13]:
align.alignto(adk_open, adk_closed, select='name CA')
adk_open.atoms.write('aligned.pdb')

In [14]:
# from_file_view = nv.show_mdanalysis(mda.Universe('aligned.pdb'))
# from_file_view

<center>
<div style='width: 800px'>
    
![from_file_view](images/aligning_structure_to_another-from_file_view.png)
    
</div>
</center>

## References

[1] Oliver Beckstein, Elizabeth&nbsp;J. Denning, Juan&nbsp;R. Perilla, and Thomas&nbsp;B. Woolf.
Zipping and <span class="bibtex-protected">Unzipping</span> of <span class="bibtex-protected">Adenylate</span> <span class="bibtex-protected">Kinase</span>: <span class="bibtex-protected">Atomistic</span> <span class="bibtex-protected">Insights</span> into the <span class="bibtex-protected">Ensemble</span> of <span class="bibtex-protected">Open</span>↔<span class="bibtex-protected">Closed</span> <span class="bibtex-protected">Transitions</span>.
<em>Journal of Molecular Biology</em>, 394(1):160–176, November 2009.
00107.
URL: <a href="https://linkinghub.elsevier.com/retrieve/pii/S0022283609011164">https://linkinghub.elsevier.com/retrieve/pii/S0022283609011164</a>, <a href="https://doi.org/10.1016/j.jmb.2009.09.009">doi:10.1016/j.jmb.2009.09.009</a>.

[2] Richard&nbsp;J. Gowers, Max Linke, Jonathan Barnoud, Tyler J.&nbsp;E. Reddy, Manuel&nbsp;N. Melo, Sean&nbsp;L. Seyler, Jan Domański, David&nbsp;L. Dotson, Sébastien Buchoux, Ian&nbsp;M. Kenney, and Oliver Beckstein.
<span class="bibtex-protected">MDAnalysis</span>: <span class="bibtex-protected">A</span> <span class="bibtex-protected">Python</span> <span class="bibtex-protected">Package</span> for the <span class="bibtex-protected">Rapid</span> <span class="bibtex-protected">Analysis</span> of <span class="bibtex-protected">Molecular</span> <span class="bibtex-protected">Dynamics</span> <span class="bibtex-protected">Simulations</span>.
<em>Proceedings of the 15th Python in Science Conference</em>, pages 98–105, 2016.
00152.
URL: <a href="https://conference.scipy.org/proceedings/scipy2016/oliver_beckstein.html">https://conference.scipy.org/proceedings/scipy2016/oliver_beckstein.html</a>, <a href="https://doi.org/10.25080/Majora-629e541a-00e">doi:10.25080/Majora-629e541a-00e</a>.

[3] Pu&nbsp;Liu, Dimitris&nbsp;K. Agrafiotis, and Douglas&nbsp;L. Theobald.
Fast determination of the optimal rotational matrix for macromolecular superpositions.
<em>Journal of Computational Chemistry</em>, pages n/a–n/a, 2009.
URL: <a href="http://doi.wiley.com/10.1002/jcc.21439">http://doi.wiley.com/10.1002/jcc.21439</a>, <a href="https://doi.org/10.1002/jcc.21439">doi:10.1002/jcc.21439</a>.

[4] Naveen Michaud-Agrawal, Elizabeth&nbsp;J. Denning, Thomas&nbsp;B. Woolf, and Oliver Beckstein.
<span class="bibtex-protected">MDAnalysis</span>: <span class="bibtex-protected">A</span> toolkit for the analysis of molecular dynamics simulations.
<em>Journal of Computational Chemistry</em>, 32(10):2319–2327, July 2011.
00778.
URL: <a href="http://doi.wiley.com/10.1002/jcc.21787">http://doi.wiley.com/10.1002/jcc.21787</a>, <a href="https://doi.org/10.1002/jcc.21787">doi:10.1002/jcc.21787</a>.

[5] Hai Nguyen, David&nbsp;A Case, and Alexander&nbsp;S Rose.
<span class="bibtex-protected">NGLview</span>–interactive molecular graphics for <span class="bibtex-protected">Jupyter</span> notebooks.
<em>Bioinformatics</em>, 34(7):1241–1242, April 2018.
00024.
URL: <a href="https://academic.oup.com/bioinformatics/article/34/7/1241/4721781">https://academic.oup.com/bioinformatics/article/34/7/1241/4721781</a>, <a href="https://doi.org/10.1093/bioinformatics/btx789">doi:10.1093/bioinformatics/btx789</a>.

[6] Douglas&nbsp;L. Theobald.
Rapid calculation of <span class="bibtex-protected">RMSDs</span> using a quaternion-based characteristic polynomial.
<em>Acta Crystallographica Section A Foundations of Crystallography</em>, 61(4):478–480, July 2005.
00127.
URL: <a href="http://scripts.iucr.org/cgi-bin/paper?S0108767305015266">http://scripts.iucr.org/cgi-bin/paper?S0108767305015266</a>, <a href="https://doi.org/10.1107/S0108767305015266">doi:10.1107/S0108767305015266</a>.